# Validate a tool against IDR data

The image (``id=6001247``) is referenced in the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388 and can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=image-6001247)(IDR).

IDR is based on [OMERO](https://www.openmicroscopy.org/omero/) so anything used during this session can be used against an OMERO server


The authors considered various packages. The objectives of this workshop:

* How to access public resource via the [Python API](https://omero-guides.readthedocs.io/en/latest/python/docs/index.html).
* How to load image data.
* How to load binary data.
* How to load Regions of Interest (ROIs) associated to an image.
* Compare ROIs submitted by the authors and the ones generated during a new snalysis.

Loading data via the API **does not** write a file on disk. If you wish to download file(s) from IDR and write them on disk, please check [Data download](https://idr.openmicroscopy.org/about/download.html).

We will analyse the data using [Stardist](https://github.com/stardist/stardist) and compare the output. Stardist was not considered by the authors. This shows how public repository can be accessed to valid tools or new algorithm.

## First connect to IDR

This is the first step to any action when using the Python API.

In [8]:
from omero.gateway import BlitzGateway
HOST = 'ws://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

True


## Load an image

We load information about the image but **not** the binary data. An image is a 5D-object (XYZCT).

In [9]:
image_id = 6001247

In [10]:
image = conn.getObject("Image", image_id)
print(image.getName())

B4_C3.tif


To get the dimension along the an axis e.g. z-axis. Do:

In [17]:
print(image.getSizeY())

210


Exercise: Print the version along the T and C axis

## Load the binary

To access the binary, we need to load the `pixels` object from the image and retrieve the plane(s)

In [13]:
pixels = image.getPrimaryPixels()

When using the API, you can only load the planes you need to analyze. This is an advantage when using analytical tools requiring planar data e.g. CellProfiler. The example below show to load one plane as a `YX` [NumPy](https://numpy.org/) array.
We load the plane ``z, t, c = 0, 0, 0 `` 

In [18]:
plane = pixels.getPlane(0, 0, 0)
# Check the size of the array using the numpy ``shape`` method.
print(plane.shape)

(210, 253)


To load multiple planes use ``pixels.getPlanes()``

In [20]:
zct_list = [(0, 0, 0), (1, 0, 0)]
planes = pixels.getPlanes(zct_list)
print(len(list(planes)))

2


## Exercise: Load the 5D image as a `TCZYX` numpy array

Using the example below. 
* Create a method `load_numpy_array` taking the image as a parameter
* Iterate over the T, C, Z axis and create a list of coordinates
* Use the ``getPlanes`` method to load the 2D numpy array.
* Use the ``numpy.stack`` method to create an 5D numpy array
* Check the dimension order of the 5D numpy array, reshape using `numpy.reshape` to create an array in the order `TCZYX`.

## Load the 5D array

Using the method created in the exercise. We load the binary data as 5D numpy array.

In [ ]:
data = load_numpy_array(image)

## Load StarDist trained model 

We use an existing trained model from StarDist. The tool was introduced during Day 1.

In [1]:
from stardist.models import StarDist2D
model_versatile = StarDist2D.from_pretrained('2D_demo')

Found model '2D_demo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


2022-05-12 10:44:12.372629: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prediction based on a default StarDist model

Normalize the input image

``model_versatile.predict_instances`` will

 * predict object probabilities and star-convex polygon distances (see model.predict if you want those)
* perform non-maximum suppression (with overlap threshold nms_thresh) for polygons above object probability threshold prob_thresh.
* render all remaining polygon instances in a label image
* return the label instances image and also the details (coordinates, etc.) of all remaining polygons

We will analyze the second channel i.e. ``c = 1``. We could have only loaded the second channel. This is left as an exercise to adjust the method above to only load the second channel.

In [ ]:
# Channel to analyze
c = 1
from csbdeep.utils import normalize
axis_norm = (0,1)
img = normalize(data[0, c, :, :, :], 1,99.8, axis=axis_norm)
results = []
results_details = []
for i in range(len(img)):
    new_labels, details = model_versatile.predict_instances(img[i])
    results_details.append(details)
    results.append(new_labels)

label_slices = numpy.array(results)

## Load the ROIs from IDR

Load the original labels in order to compare them with the StarDist ones Original labels have been saved as mask.

In [ ]:
roi_service = conn.getRoiService()
result = roi_service.findByImage(image_id, None)

shapes = []
for roi in result.rois:
    shapes.append(roi.copyShapes())

To compare the labels, we need to convert the masks into a NumPy array. 

In [ ]:
# from omero_zarr import masks
dims = (image.getSizeT(), image.getSizeC(), image.getSizeZ(), image.getSizeY(), image.getSizeX())
saver = masks.MaskSaver(None, image, numpy.int64)
labels, fillColors, properties = saver.masks_to_labels(shapes, mask_shape=dims)

## Compare labels

Display the original labels and the labels based on StarDist prediction side-by-side

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def update(z=0):
    c = 1
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(data[0, c, z, :, :], cmap='jet')
    try:
        plt.imshow(labels[0, c, z, :, :], cmap='gray', alpha=0.5)
    except Exception:
        print(z)
    plt.subplot(122)
    plt.imshow(data[0, c, z, :, :], cmap='gray')
    plt.imshow(label_slices[z, :, :], cmap='jet', alpha=0.5)
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, z= widgets.IntSlider(value=1, min=0, max=data.shape[2]-1, step=1, description="Select Z", continuous_update=False))


## Close the connection

In [ ]:
conn.close()

## Exercise: Load the data and the labels from S3 

The original image has been converted into Zarr and is available on S3.
Using approach presented during Day 3, retrieve the data and labels from S3 instead of IDR and compare the output with the StarDist labels.